In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

from sklearn import metrics

import scripts.blosc_interface as bi
import scripts.corpus_split as cs

https://stackoverflow.com/questions/26826002/adding-words-to-stop-words-list-in-tfidfvectorizer-in-sklearn
https://stackoverflow.com/questions/50756085/how-to-print-the-progress-of-a-list-comprehension-in-python
https://stackoverflow.com/questions/57983431/whats-the-most-space-efficient-way-to-compress-serialized-python-data

In [2]:
full_corpus = bi.blosc_read("./data/tokenized_corpus.dat")

In [3]:
x_train, x_test, y_train, y_test = cs.corpus_split(full_corpus)


Splitting the Data into Train and Test


In [4]:
log_reg_cv_results = bi.blosc_read("./data/log_grid_search_result.dat")
log_reg_top5 = log_reg_cv_results.query("rank_test_score <= 5")
log_reg_top5

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_vect__ngram_range,param_vect__max_features,param_svd__n_components,param_log_clf__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
9,64.123026,3.714061,6.076129,1.974486,"(1, 2)",400,30,10000.0,"{'vect__ngram_range': (1, 2), 'vect__max_featu...",0.848319,0.839127,0.827636,0.837690,0.850043,0.840563,0.008093,5
14,58.169902,0.920133,4.012292,0.195800,"(1, 2)",400,60,21.544347,"{'vect__ngram_range': (1, 2), 'vect__max_featu...",0.867567,0.857512,0.852054,0.846883,0.873600,0.859523,0.009823,1
18,12.827075,0.430348,2.242276,0.205860,"(1, 1)",400,60,21.544347,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.863832,0.851767,0.846309,0.844872,0.865843,0.854525,0.008752,3
35,11.156855,0.467028,2.069154,0.159691,"(1, 1)",300,45,10000.0,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.852054,0.844585,0.839989,0.833668,0.859523,0.845964,0.009052,4
38,12.062435,0.280006,2.068953,0.173405,"(1, 1)",400,60,10000.0,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.864694,0.854352,0.850618,0.846596,0.867280,0.856708,0.008005,2


In [5]:
from sklearn.linear_model import LogisticRegression
# Loading Up the Same Pipeline as Random Search
# Updated Values to Optimized
log_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ("svd", TruncatedSVD()),
            ('log_clf', LogisticRegression())
        ])

# Fitting the Optimized Log Reg Model
log_clf.fit(x_train, y_train)

# Getting Predictions on the testset
predicted = log_clf.predict(x_test)
np.mean(predicted == y_test)

# Printing the results
print(metrics.classification_report(y_test, predicted))

metrics.confusion_matrix(y_test, predicted)


              precision    recall  f1-score   support

   left-wing       0.55      0.37      0.44      2097
  right-wing       0.55      0.72      0.63      2255

    accuracy                           0.55      4352
   macro avg       0.55      0.55      0.53      4352
weighted avg       0.55      0.55      0.54      4352



array([[ 780, 1317],
       [ 631, 1624]])

In [6]:
# Loading Up the Same Pipeline as Random Search
# Updated Values to Optimized
log_opt_clf = Pipeline([
            ('vect', CountVectorizer(max_features=400, ngram_range=(1,2))),
            ('tfidf', TfidfTransformer()),
            ("svd", TruncatedSVD(n_components=60)),
            ('log_clf', LogisticRegression(penalty="l1", solver = 'saga', C=21.544347, max_iter=1000)), # this needs to be a different solver for LASSO
        ])

# Fitting the Optimized Log Reg Model
log_opt_clf.fit(x_train, y_train)

# Getting Predictions on the testset
predicted = log_opt_clf.predict(x_test)
np.mean(predicted == y_test)

# Printing the results
print(metrics.classification_report(y_test, predicted))

metrics.confusion_matrix(y_test, predicted)

              precision    recall  f1-score   support

   left-wing       0.87      0.84      0.85      2097
  right-wing       0.85      0.88      0.87      2255

    accuracy                           0.86      4352
   macro avg       0.86      0.86      0.86      4352
weighted avg       0.86      0.86      0.86      4352



array([[1758,  339],
       [ 272, 1983]])

https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html#sphx-glr-auto-examples-compose-plot-digits-pipe-py

In [31]:
log_opt_clf[0].vocabulary_

{'past': 261,
 'two': 371,
 'years': 397,
 'whether': 386,
 'united': 373,
 'states': 345,
 'housing': 158,
 'would': 395,
 'financial': 116,
 'economy': 98,
 'prices': 281,
 'first': 118,
 'however': 159,
 'may': 217,
 'number': 244,
 'process': 285,
 'home': 156,
 'income': 165,
 'families': 112,
 'five': 119,
 'several': 332,
 'high': 153,
 'new': 238,
 'york': 399,
 'city': 54,
 'washington': 382,
 '000': 0,
 'according': 4,
 'opportunity': 251,
 'national': 231,
 'less': 200,
 'percent': 265,
 'access': 3,
 'house': 157,
 'areas': 24,
 'much': 228,
 'media': 219,
 'nation': 230,
 'real': 295,
 'data': 81,
 'country': 73,
 'still': 346,
 'american': 17,
 'second': 322,
 'use': 376,
 'development': 94,
 'impact': 161,
 'growth': 147,
 'people': 263,
 'market': 216,
 'report': 304,
 'problem': 283,
 'could': 71,
 'public': 289,
 'policy': 272,
 'future': 134,
 'create': 78,
 'low': 208,
 'back': 28,
 'although': 14,
 'cost': 69,
 'local': 206,
 'increased': 167,
 'government': 142,
 

In [35]:
log_opt_clf[2].explained_variance_

array([0.00999342, 0.02655163, 0.02587907, 0.02104172, 0.01868057,
       0.01556065, 0.01453942, 0.0141053 , 0.01339169, 0.01308897,
       0.01171164, 0.01129167, 0.01045946, 0.01028872, 0.00970334,
       0.00897715, 0.00843319, 0.00827448, 0.00811224, 0.00785614,
       0.0074517 , 0.00715055, 0.00712232, 0.00679074, 0.00656477,
       0.00647838, 0.00628882, 0.00617062, 0.00594708, 0.00585181,
       0.0055752 , 0.00553885, 0.00534434, 0.00523186, 0.00507057,
       0.00498511, 0.00495369, 0.00473667, 0.00470005, 0.00458031,
       0.0045251 , 0.00449452, 0.00426861, 0.00418954, 0.00411882,
       0.00404057, 0.00403422, 0.00399984, 0.00389708, 0.00385365,
       0.0037711 , 0.00372043, 0.0036199 , 0.00357318, 0.00356906,
       0.00354796, 0.00339719, 0.003383  , 0.00334645, 0.00325617])

In [8]:
svc_reg_cv_results = bi.blosc_read("./data/svc_grid_search_result.dat")
svc_top5 = svc_reg_cv_results.query("rank_test_score <= 5")
svc_top5

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_vect__ngram_range,param_vect__max_features,param_svd__n_components,param_svc_clf__penalty,param_svc_clf__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,9.538747,0.252239,1.554701,0.083288,"(1, 1)",400,45,l2,21.544347,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.857799,0.847745,0.841712,0.840563,0.862396,0.850043,0.008687,2
12,8.757007,0.317715,1.550521,0.073838,"(1, 1)",200,60,l2,0.046416,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.846021,0.832232,0.829934,0.826774,0.843149,0.835622,0.007575,5
14,40.221779,1.873627,3.265838,0.243435,"(1, 2)",400,45,l2,0.046416,"{'vect__ngram_range': (1, 2), 'vect__max_featu...",0.860672,0.847745,0.841999,0.840850,0.860385,0.850330,0.008649,1
24,10.407523,0.389623,1.543918,0.103774,"(1, 1)",300,60,l2,21.544347,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.853778,0.847170,0.849756,0.838839,0.860672,0.850043,0.007222,2
39,42.016662,1.504122,2.994632,0.116347,"(1, 2)",200,60,l2,21.544347,"{'vect__ngram_range': (1, 2), 'vect__max_featu...",0.842861,0.833668,0.832519,0.822465,0.848607,0.836024,0.009019,4


In [9]:
from sklearn.svm import LinearSVC

svc_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ("svd", TruncatedSVD()),
            ('svc_clf', LinearSVC()),
        ])

svc_clf.fit(x_train, y_train)

predicted = svc_clf.predict(x_test)
np.mean(predicted == y_test)

print(metrics.classification_report(y_test, predicted))

metrics.confusion_matrix(y_test, predicted)

              precision    recall  f1-score   support

   left-wing       0.55      0.37      0.45      2097
  right-wing       0.55      0.72      0.63      2255

    accuracy                           0.55      4352
   macro avg       0.55      0.55      0.54      4352
weighted avg       0.55      0.55      0.54      4352



array([[ 783, 1314],
       [ 632, 1623]])

In [10]:
svc_opt_clf = Pipeline([
            ('vect', CountVectorizer(max_features=400, ngram_range=(1,2))),
            ('tfidf', TfidfTransformer()),
            ("svd", TruncatedSVD(n_components=45)),
            ('svc_clf', LinearSVC(penalty = "l2", C = 0.046415)),
        ])

svc_opt_clf.fit(x_train, y_train)

predicted = svc_opt_clf.predict(x_test)
np.mean(predicted == y_test)

print(metrics.classification_report(y_test, predicted))

metrics.confusion_matrix(y_test, predicted)

              precision    recall  f1-score   support

   left-wing       0.86      0.83      0.84      2097
  right-wing       0.85      0.88      0.86      2255

    accuracy                           0.85      4352
   macro avg       0.85      0.85      0.85      4352
weighted avg       0.85      0.85      0.85      4352



array([[1735,  362],
       [ 276, 1979]])

In [30]:
svc_opt_clf[0].vocabulary_

{'past': 261,
 'two': 371,
 'years': 397,
 'whether': 386,
 'united': 373,
 'states': 345,
 'housing': 158,
 'would': 395,
 'financial': 116,
 'economy': 98,
 'prices': 281,
 'first': 118,
 'however': 159,
 'may': 217,
 'number': 244,
 'process': 285,
 'home': 156,
 'income': 165,
 'families': 112,
 'five': 119,
 'several': 332,
 'high': 153,
 'new': 238,
 'york': 399,
 'city': 54,
 'washington': 382,
 '000': 0,
 'according': 4,
 'opportunity': 251,
 'national': 231,
 'less': 200,
 'percent': 265,
 'access': 3,
 'house': 157,
 'areas': 24,
 'much': 228,
 'media': 219,
 'nation': 230,
 'real': 295,
 'data': 81,
 'country': 73,
 'still': 346,
 'american': 17,
 'second': 322,
 'use': 376,
 'development': 94,
 'impact': 161,
 'growth': 147,
 'people': 263,
 'market': 216,
 'report': 304,
 'problem': 283,
 'could': 71,
 'public': 289,
 'policy': 272,
 'future': 134,
 'create': 78,
 'low': 208,
 'back': 28,
 'although': 14,
 'cost': 69,
 'local': 206,
 'increased': 167,
 'government': 142,
 

In [36]:
svc_opt_clf[2].explained_variance_

array([0.00999342, 0.02655163, 0.02587907, 0.02104172, 0.01868057,
       0.01556065, 0.01453942, 0.0141053 , 0.01339169, 0.01308897,
       0.01171164, 0.01129166, 0.01045944, 0.0102887 , 0.00970333,
       0.00897707, 0.00843297, 0.00827445, 0.00811203, 0.00785521,
       0.00745135, 0.0071496 , 0.00712206, 0.00678999, 0.00656446,
       0.00647703, 0.00628619, 0.0061665 , 0.00594519, 0.00584741,
       0.0055662 , 0.00552201, 0.00533001, 0.00522641, 0.00503176,
       0.00497706, 0.00491041, 0.00470204, 0.00464394, 0.00454361,
       0.00448433, 0.00446361, 0.00420491, 0.00409596, 0.0040299 ])

In [11]:
knn_reg_cv_results = bi.blosc_read("./data/knn_grid_search_result.dat")
knn_top5 = knn_reg_cv_results.query("rank_test_score <= 5")
knn_top5

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_vect__ngram_range,param_vect__max_features,param_svd__n_components,param_knn_clf__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,7.708155,0.123026,2.061593,0.065220,"(1, 1)",400,60,5,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.858661,0.861247,0.856076,0.850618,0.863545,0.858029,0.004470,1
7,38.017391,0.497011,3.561517,0.061605,"(1, 2)",300,45,6,"{'vect__ngram_range': (1, 2), 'vect__max_featu...",0.845734,0.855214,0.853203,0.838552,0.848319,0.848205,0.005889,4
10,8.183979,0.187956,2.308006,0.287278,"(1, 1)",300,45,5,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.849181,0.852054,0.853778,0.843149,0.845159,0.848664,0.004015,3
27,38.280288,0.235090,3.726631,0.112006,"(1, 2)",400,45,6,"{'vect__ngram_range': (1, 2), 'vect__max_featu...",0.846021,0.851767,0.858949,0.851767,0.856938,0.853088,0.004530,2
35,8.004029,0.397554,2.160517,0.036752,"(1, 1)",300,60,6,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.852629,0.849469,0.852341,0.836541,0.842861,0.846768,0.006206,5


In [12]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ("svd", TruncatedSVD()),
        ('knn_clf', KNeighborsClassifier()),
    ])

knn_clf.fit(x_train, y_train)

predicted = knn_clf.predict(x_test)
np.mean(predicted == y_test)

print(metrics.classification_report(y_test, predicted))

metrics.confusion_matrix(y_test, predicted)

              precision    recall  f1-score   support

   left-wing       0.54      0.53      0.53      2097
  right-wing       0.57      0.58      0.58      2255

    accuracy                           0.56      4352
   macro avg       0.55      0.55      0.55      4352
weighted avg       0.55      0.56      0.55      4352



array([[1103,  994],
       [ 942, 1313]])

In [13]:
knn_opt_clf = Pipeline([
        ('vect', CountVectorizer(max_features=400, ngram_range=(1,1))),
        ('tfidf', TfidfTransformer()),
        ("svd", TruncatedSVD(n_components=60)),
        ('knn_clf', KNeighborsClassifier(n_neighbors = 5)),
    ])

knn_opt_clf.fit(x_train, y_train)

predicted = knn_opt_clf.predict(x_test)
np.mean(predicted == y_test)

print(metrics.classification_report(y_test, predicted))

metrics.confusion_matrix(y_test, predicted)

              precision    recall  f1-score   support

   left-wing       0.88      0.82      0.85      2097
  right-wing       0.84      0.90      0.87      2255

    accuracy                           0.86      4352
   macro avg       0.86      0.86      0.86      4352
weighted avg       0.86      0.86      0.86      4352



array([[1725,  372],
       [ 235, 2020]])

In [29]:
knn_opt_clf[0].vocabulary_

{'past': 260,
 'two': 372,
 'years': 397,
 'whether': 386,
 'united': 374,
 'states': 346,
 'housing': 158,
 'would': 395,
 'financial': 117,
 'economy': 98,
 'prices': 280,
 'first': 119,
 'however': 159,
 'may': 217,
 'number': 243,
 'process': 284,
 'home': 156,
 'income': 165,
 'families': 113,
 'five': 120,
 'several': 333,
 'high': 153,
 'new': 238,
 'york': 399,
 'city': 55,
 'washington': 382,
 '000': 0,
 'according': 4,
 'opportunity': 250,
 'national': 231,
 'less': 200,
 'percent': 264,
 'access': 3,
 'house': 157,
 'areas': 24,
 'much': 228,
 'media': 219,
 'nation': 230,
 'real': 294,
 'data': 81,
 'country': 74,
 'still': 347,
 'american': 17,
 'second': 322,
 'use': 376,
 'development': 94,
 'impact': 161,
 'growth': 149,
 'people': 262,
 'market': 216,
 'report': 304,
 'problem': 282,
 'could': 72,
 'public': 288,
 'policy': 271,
 'future': 135,
 'create': 78,
 'low': 208,
 'back': 28,
 'although': 14,
 'cost': 70,
 'local': 206,
 'increased': 167,
 'government': 143,
 

In [37]:
knn_opt_clf[2].explained_variance_

array([0.01001954, 0.02633636, 0.02578821, 0.02071068, 0.01865065,
       0.01544238, 0.01424899, 0.01390376, 0.01327717, 0.01311909,
       0.01171396, 0.01128584, 0.01031973, 0.0101792 , 0.00910659,
       0.00875136, 0.00846611, 0.00826602, 0.00788312, 0.00771776,
       0.00740674, 0.00697513, 0.00691148, 0.00673303, 0.00657712,
       0.0064485 , 0.00618466, 0.00614202, 0.00590764, 0.00582941,
       0.00555531, 0.005446  , 0.00532617, 0.00524535, 0.00500309,
       0.00496463, 0.00492392, 0.00473881, 0.00468373, 0.00459165,
       0.00451527, 0.00436548, 0.00425258, 0.00417394, 0.00411131,
       0.00405586, 0.00401627, 0.00389464, 0.00386563, 0.00384886,
       0.00377784, 0.00369077, 0.00360974, 0.00360824, 0.00351653,
       0.00346191, 0.00343374, 0.00337285, 0.00329714, 0.0032845 ])

In [14]:
forest_cv_results = bi.blosc_read("./data/forest_grid_search_result.dat")
forest_top5 = forest_cv_results.query("rank_test_score <= 5")
forest_top5

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_vect__ngram_range,param_vect__max_features,param_svd__n_components,param_forest_clf__max_features,param_forest_clf__max_depth,param_forest_clf__criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,44.538229,0.430227,3.097426,0.044942,"(1, 2)",400,60,log2,15,entropy,"{'vect__ngram_range': (1, 2), 'vect__max_featu...",0.874749,0.860959,0.860959,0.851479,0.871876,0.864005,0.008400,3
11,11.257121,0.414312,1.563059,0.056807,"(1, 1)",400,60,log2,15,gini,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.873025,0.863258,0.859523,0.851192,0.868716,0.863143,0.007545,5
25,44.256173,0.485781,3.049271,0.055533,"(1, 2)",400,60,log2,35,entropy,"{'vect__ngram_range': (1, 2), 'vect__max_featu...",0.871014,0.859810,0.862396,0.856650,0.879920,0.865958,0.008457,1
29,43.861682,0.323839,3.080161,0.093970,"(1, 2)",400,30,sqrt,25,entropy,"{'vect__ngram_range': (1, 2), 'vect__max_featu...",0.869578,0.859236,0.862683,0.856076,0.872738,0.864062,0.006240,2
32,12.100364,0.267698,1.614859,0.105144,"(1, 1)",400,45,sqrt,15,gini,"{'vect__ngram_range': (1, 1), 'vect__max_featu...",0.870152,0.860098,0.855214,0.856938,0.874174,0.863315,0.007501,4


In [15]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ("svd", TruncatedSVD()),
        ('forest_clf', RandomForestClassifier()),
    ])

forest_clf.fit(x_train, y_train)

predicted = forest_clf.predict(x_test)
np.mean(predicted == y_test)

print(metrics.classification_report(y_test, predicted))

metrics.confusion_matrix(y_test, predicted)

              precision    recall  f1-score   support

   left-wing       0.54      0.54      0.54      2097
  right-wing       0.57      0.58      0.58      2255

    accuracy                           0.56      4352
   macro avg       0.56      0.56      0.56      4352
weighted avg       0.56      0.56      0.56      4352



array([[1135,  962],
       [ 955, 1300]])

In [16]:
forest_opt_clf = Pipeline([
        ('vect', CountVectorizer(max_features=400, ngram_range=(1,2))),
        ('tfidf', TfidfTransformer()),
        ("svd", TruncatedSVD(n_components=60)),
        ('forest_clf', RandomForestClassifier(max_features="log2",max_depth=35,
        criterion="entropy")), 
    ])

forest_opt_clf.fit(x_train, y_train)

predicted = forest_opt_clf.predict(x_test)
np.mean(predicted == y_test)

print(metrics.classification_report(y_test, predicted))

metrics.confusion_matrix(y_test, predicted)

              precision    recall  f1-score   support

   left-wing       0.89      0.86      0.87      2097
  right-wing       0.87      0.90      0.88      2255

    accuracy                           0.88      4352
   macro avg       0.88      0.88      0.88      4352
weighted avg       0.88      0.88      0.88      4352



array([[1794,  303],
       [ 225, 2030]])

In [28]:
forest_opt_clf[0].vocabulary_

{'past': 261,
 'two': 371,
 'years': 397,
 'whether': 386,
 'united': 373,
 'states': 345,
 'housing': 158,
 'would': 395,
 'financial': 116,
 'economy': 98,
 'prices': 281,
 'first': 118,
 'however': 159,
 'may': 217,
 'number': 244,
 'process': 285,
 'home': 156,
 'income': 165,
 'families': 112,
 'five': 119,
 'several': 332,
 'high': 153,
 'new': 238,
 'york': 399,
 'city': 54,
 'washington': 382,
 '000': 0,
 'according': 4,
 'opportunity': 251,
 'national': 231,
 'less': 200,
 'percent': 265,
 'access': 3,
 'house': 157,
 'areas': 24,
 'much': 228,
 'media': 219,
 'nation': 230,
 'real': 295,
 'data': 81,
 'country': 73,
 'still': 346,
 'american': 17,
 'second': 322,
 'use': 376,
 'development': 94,
 'impact': 161,
 'growth': 147,
 'people': 263,
 'market': 216,
 'report': 304,
 'problem': 283,
 'could': 71,
 'public': 289,
 'policy': 272,
 'future': 134,
 'create': 78,
 'low': 208,
 'back': 28,
 'although': 14,
 'cost': 69,
 'local': 206,
 'increased': 167,
 'government': 142,
 

In [38]:
forest_opt_clf[2].explained_variance_

array([0.00999342, 0.02655163, 0.02587907, 0.02104172, 0.01868057,
       0.01556065, 0.01453942, 0.0141053 , 0.01339169, 0.01308897,
       0.01171164, 0.01129167, 0.01045946, 0.01028872, 0.00970334,
       0.00897714, 0.00843318, 0.0082745 , 0.00811222, 0.00785617,
       0.00745173, 0.00715057, 0.00712232, 0.00679096, 0.00656512,
       0.00647889, 0.00628874, 0.00617092, 0.00594727, 0.00585134,
       0.00557567, 0.00553865, 0.00534393, 0.00523238, 0.00506958,
       0.00498049, 0.0049511 , 0.00473881, 0.0047036 , 0.00457587,
       0.0045227 , 0.00448466, 0.00427141, 0.00418954, 0.00412463,
       0.00406127, 0.00403277, 0.00397749, 0.00390995, 0.00385661,
       0.00378418, 0.00374152, 0.00365741, 0.00360852, 0.0035587 ,
       0.00345572, 0.00344809, 0.00339067, 0.0033253 , 0.00324853])